In [1]:
import pyltp
import xlwt
import xlrd
import openpyxl
import self as self
import numpy as np
import pandas as pd
import xlsxwriter as xlsxwriter
from numpy import angle
from pyltp import Segmentor
from pyltp import SentenceSplitter
from pyltp import Postagger

#读取文件，文件读取函数
def read_file(filename):
    with  open(filename, 'r',encoding='utf-8')as f:
        text = f.read()
        #返回list类型数据
        text = text.split('\n')
    return text
  
def read_excel(filename):
    pl = xlrd.open_workbook(filename)
    sheet = pl.sheet_by_index(0)

    text = [str(sheet.cell(i, 1).value) for i in range(sheet.nrows)]

    return text

#将数据写入文件中
def write_data(filename,data):
    with open(filename,'a',encoding='utf-8')as f:
        f.write(str(data))
 
def write_excel_header():
    pl = openpyxl.Workbook()
    sheet = pl.active
    sheet['A1'] = "情感分值"
    sheet['B1'] = "情感倾向"
    sheet['C1'] = "情感分析文本"
    return pl

def write_excel(pl, data1, data2, data3):
    sheet = pl.active
    sheet.cell(row=sheet.max_row + 1, column=1, value=data1)
    sheet.cell(row=sheet.max_row, column=2, value=data2)
    sheet.cell(row=sheet.max_row, column=3, value=data3)

def save_excel(pl, filename):
    pl.save(filename)

# 文本分句
def cut_sentence(text):
    sentences = SentenceSplitter.split(text)
    sentence_list = [w for w in sentences]
    return sentence_list
  
#文本分词
def tokenize(sentence):
    #加载模型
    segmentor = Segmentor()  # 初始化实例
    # 加载模型
    segmentor.load(r'D:\qinggan\ltp_data_v3.4.0\ltp_data_v3.4.0\cws.model')
    # 产生分词，segment分词函数
    words = segmentor.segment(sentence)
    words = list(words)
    # 释放模型
    segmentor.release()
    return words
  
#词性标注
def postagger(words):
    # 初始化实例
    postagger = Postagger()
    # 加载模型
    postagger.load(r'D:\qinggan\ltp_data_v3.4.0\ltp_data_v3.4.0\pos.model')
    # 词性标注
    postags = postagger.postag(words)
    # 释放模型
    postagger.release()
    #返回list
    postags = [i for i in postags]
    return postags
 
# 分词，词性标注，词和词性构成一个元组
def intergrad_word(words,postags):
    #拉链算法，两两匹配
    pos_list = zip(words,postags)
    pos_list = [ w for w in pos_list]
    return pos_list
 
#去停用词函数
def del_stopwords(words):
    # 读取停用词表
    stopwords = read_file(r"D:\qinggan\stopwords.txt")
    # 去除停用词后的句子
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words
 
# 获取六种权值的词，根据要求返回list，这个函数是为了配合Django的views下的函数使用
def weighted_value(request):
    result_dict = []
    if request == "one":
        result_dict = read_file(r"D:\qinggan\degree_dict\most.txt")
    elif request == "two":
        result_dict = read_file(r"D:\qinggan\degree_dict\very.txt")
    elif request == "three":
        result_dict = read_file(r"D:\qinggan\degree_dict\more.txt")
    elif request == "four":
        result_dict = read_file(r"D:\qinggan\degree_dict\ish.txt")
    elif request == "five":
        result_dict = read_file(r"D:\qinggan\degree_dict\insufficiently.txt")
    elif request == "six":
        result_dict = read_file(r"D:\qinggan\degree_dict\inverse.txt")
    elif request == 'posdict':
        result_dict = read_file(r"D:\qinggan\emotion_dict\posdict.txt")
    elif request == 'negdict':
        result_dict = read_file(r"D:\qinggan\emotion_dict\negdict.txt")
    else:
        pass
    return result_dict
  
print("reading sentiment dict .......")
#读取情感词典
posdict = weighted_value('posdict')
negdict = weighted_value('negdict')
# 读取程度副词词典
# 权值为2
mostdict = weighted_value('one')
# 权值为1.75
verydict = weighted_value('two')
# 权值为1.50
moredict = weighted_value('three')
# 权值为1.25
ishdict = weighted_value('four')
# 权值为0.25
insufficientdict = weighted_value('five')
# 权值为-1
inversedict = weighted_value('six')
  
#程度副词处理，对不同的程度副词给予不同的权重
def match_adverb(word,sentiment_value):
    #最高级权重为
    if word in mostdict:
        sentiment_value *= 8
    #比较级权重
    elif word in verydict:
        sentiment_value *= 6
    #比较级权重
    elif word in moredict:
        sentiment_value *= 4
    #轻微程度词权重
    elif word in ishdict:
        sentiment_value *= 2
    #相对程度词权重
    elif word in insufficientdict:
        sentiment_value *= 0.5
    #否定词权重
    elif word in inversedict:
        sentiment_value *= -1
    else:
        sentiment_value *= 1
    return sentiment_value
 
#对每一条评论打分
def single_sentiment_score(text_sent):
    sentiment_scores = []
    #对单条评论分句
    sentences = cut_sentence(text_sent)
    for sent in sentences:
        #查看分句结果
        # print('分句：',sent)
        #分词
        words = tokenize(sent)
        seg_words = del_stopwords(words)
        #i，s 记录情感词和程度词出现的位置
        i = 0   #记录扫描到的词位子
        s = 0   #记录情感词的位置
        poscount = 0 #记录积极情感词数目
        negcount = 0 #记录消极情感词数目
        #逐个查找情感词
        for word in seg_words:
            #如果为积极词
            if word in posdict:
                poscount += 1  #情感词数目加1
            #在情感词前面寻找程度副词
                for w in seg_words[s:i]:
                    poscount = match_adverb(w,poscount)
                s = i+1 #记录情感词位置
            # 如果是消极情感词
            elif word in negdict:
                negcount +=1
                for w in seg_words[s:i]:
                    negcount = match_adverb(w,negcount)
                s = i+1
            #如果结尾为感叹号或者问号，表示句子结束，并且倒序查找感叹号前的情感词，权重+4
            elif word =='!' or  word =='！' or word =='?' or word == '？':
                for w2 in seg_words[::-1]:
                    #如果为积极词，poscount+2
                    if w2 in posdict:
                        poscount += 4
                        break
                    #如果是消极词，negcount+2
                    elif w2 in negdict:
                        negcount += 4
                        break
            i += 1 #定位情感词的位置
        #计算情感值
        sentiment_score = poscount - negcount
        sentiment_scores.append(sentiment_score)
        #查看每一句的情感值
        # print('分句分值：',sentiment_score)
    sentiment_sum = 0
    for s in sentiment_scores:
        #计算出一条评论的总得分
        sentiment_sum +=s
    return sentiment_sum

# 分析test_data.txt 中的所有评论，返回一个列表，列表中元素为（分值，评论）元组
def run_score(contents):
    # 待处理数据
    scores_list = []
    for content in contents:
        if content !='':
            score = single_sentiment_score(content)  # 对每条评论调用函数求得打分
            scores_list.append((score, content)) # 形成（分数，评论）元组
    return scores_list
 
#主程序
if __name__ == '__main__':
    print('Processing........')
    sentences = read_file(r'C:\Users\liujinfeng\Desktop\毕业论文\素材\股票评论数据\zsshpl.txt')
    scores = run_score(sentences)
    #人工标注情感词典
    man_sentiment = read_file(r'D:\qinggan\result_data\人工情感标注.txt')
    al_sentiment = []
    for score in scores:
        print('情感分值：',score[0])
        if score[0] < 0:
            print('情感倾向：消极')
            s = '消极'
        elif score[0] == 0:
            print('情感倾向：中性')
            s = '中性'
        else:
            print('情感倾向：积极')
            s = '积极'
        al_sentiment.append(s)
        print('情感分析文本：',score[1])
    i = 0
    
#写入文件中
pl = write_excel_header()
for score in scores:
    if score[0] < 0:
        write_excel(pl, str(score[0]), "消极", str(score[1]))
    elif score[0] == 0:
        write_excel(pl, str(score[0]), "中性", str(score[1]))
    else:
        write_excel(pl, str(score[0]), "积极", str(score[1]))

    i += 1
save_excel(pl, r'C:\Users\liujinfeng\Desktop\毕业论文\素材\股票评论数据\zsshplcl.xlsx')

print('succeed.......')


reading sentiment dict .......
Processing........
情感分值： 0
情感倾向：中性
情感分析文本： title
情感分值： 1
情感倾向：积极
情感分析文本： 已经连继一个月，日成交万亿以上，券商3季度利润爆涨
情感分值： 1
情感倾向：积极
情感分析文本： 别追涨
情感分值： 1
情感倾向：积极
情感分析文本： 今日这些个股异动主力青睐券商、抛售有色股，振幅大
情感分值： -1
情感倾向：消极
情感分析文本： 看到这么多看空的，我就安心了
情感分值： 0
情感倾向：中性
情感分析文本： 百亿基金经理大瓜…
情感分值： -1
情感倾向：消极
情感分析文本： 明天继续等，然后看明天情况再说后天的策略！
情感分值： 1
情感倾向：积极
情感分析文本： 起码是短线机会
情感分值： -2
情感倾向：消极
情感分析文本： 二十日线死死压制
情感分值： 0
情感倾向：中性
情感分析文本： 速度中国电信联系张庭
情感分值： -3
情感倾向：消极
情感分析文本： 一日游而已，多头最后的疯狂。明天开始继续天天下跌。
情感分值： 0
情感倾向：中性
情感分析文本： 鸡狗光拉指数有用吗
情感分值： -1
情感倾向：消极
情感分析文本： 这周能不能突破月线并站稳了
情感分值： 0
情感倾向：中性
情感分析文本： 半个小时拉了1.5%，明天出货吗？
情感分值： 1
情感倾向：积极
情感分析文本： 全部拉的底部游戏股，基建水泥，中字头，持续不了几天
情感分值： -3
情感倾向：消极
情感分析文本： 基金鸡狗在作死。宁组合矛指数破灭，价值回归。
情感分值： -1
情感倾向：消极
情感分析文本： 今天三傻启动了
情感分值： 1
情感倾向：积极
情感分析文本： 吃完多方，吃空方，再吃多方，
情感分值： 0
情感倾向：中性
情感分析文本： 有戏
情感分值： 3
情感倾向：积极
情感分析文本： 再不发力，真要被标普500追上了
情感分值： 1
情感倾向：积极
情感分析文本： 沪深300主力IF2109：昨日月线已收官，收取的一根上下
情感分值： 1
情感倾向：积极
情感分析文本： A股机会早知道：八部门联合发布《深化医疗服务价格
情感分值： 2
情感倾向：积极
情感分析文本： 锂矿概念掀起涨停潮机构、北向资金大举扫货龙头股
情感分值： 0
情感倾向：中性


情感分析文本： 今天市场整体表现比较低迷。早盘低开之后指数开始了
情感分值： 0
情感倾向：中性
情感分析文本： 有锂走遍天下，最后的jm赌局，2021.8.5星期四18:18
情感分值： 2
情感倾向：积极
情感分析文本： 可惜，昨天尾盘开空后，今天没有15分钟顶部背离，加不
情感分值： 1
情感倾向：积极
情感分析文本： 果然尾盘要杀，冲高出了，过几天再玩，有的是机会的。
情感分值： 0
情感倾向：中性
情感分析文本： 我命由我不由天，拉满杠杠继续干！
情感分值： 0
情感倾向：中性
情感分析文本： 今日不能站上MA20就完了
情感分值： 2
情感倾向：积极
情感分析文本： 买军工股，主升浪开始发力突破
情感分值： 2
情感倾向：积极
情感分析文本： 涨时蜗牛，每次想等等，等来的都是断崖式的大跌，最后
情感分值： 0
情感倾向：中性
情感分析文本： 白酒误国，科技兴邦
情感分值： 6
情感倾向：积极
情感分析文本： 股指多么漂亮的多头排列啊。我也醉了
情感分值： 6
情感倾向：积极
情感分析文本： 股指多么漂亮的多头排列啊。我也醉了
情感分值： 5
情感倾向：积极
情感分析文本： 体育产业基金，有机会么？
情感分值： 0
情感倾向：中性
情感分析文本： 当代股民的现状......
情感分值： 2
情感倾向：积极
情感分析文本： 今天市场低开回升延续反弹，上证收在最高点3477，缩量
情感分值： 0
情感倾向：中性
情感分析文本： YYDS
情感分值： -1
情感倾向：消极
情感分析文本： 仓位4成，止损3220，今天可以的是没能有效突破3240，
情感分值： -1
情感倾向：消极
情感分析文本： 跌就补涨就卖[滴汗]来嘛
情感分值： -9
情感倾向：消极
情感分析文本： 还未安全位，还要探低1次
情感分值： -1
情感倾向：消极
情感分析文本： 明天又得跌了。
情感分值： 1
情感倾向：积极
情感分析文本： 120和90分钟高九形成！其中120高九是多市场共振！明天
情感分值： 2
情感倾向：积极
情感分析文本： 从昨天开始30分钟线就顶背离了，大盘就是不跌，真够顽
情感分值： -1
情感倾向：消极
情感分析文本： 打错字，是5分钟顶部背离！
情感分值： -2
情感倾向：消极
情感分析文本： 五分钟钝化了！这个五分钟底部背离形成时开空！
情感分值： 0
情感倾向

情感倾向：中性
情感分析文本： 北向带节奏
情感分值： -1
情感倾向：消极
情感分析文本： 开始看空
情感分值： -2
情感倾向：消极
情感分析文本： 今天止损了，前两次也都是止损，这是连续第三次了，这
情感分值： 1
情感倾向：积极
情感分析文本： 下半年 沪深300 走势预测
情感分值： 2
情感倾向：积极
情感分析文本： 日线绿柱在0轴下放大了！空仓！今天尾盘没空的，可以
情感分值： 0
情感倾向：中性
情感分析文本： 这两只是目前市场唯二可以翻倍的股票。601919业绩翻52
情感分值： 1
情感倾向：积极
情感分析文本： 己准备好了接盘，快跌到3300
情感分值： 0
情感倾向：中性
情感分析文本： 又跌了，北向资金狂卖，汽车/新能源相关扑街，平安又
情感分值： -2
情感倾向：消极
情感分析文本： 北向资金今天出的有点凶啊！
情感分值： 3
情感倾向：积极
情感分析文本： 300567实现了国产芯片的重大突破，业绩翻9倍，预计股
情感分值： -2
情感倾向：消极
情感分析文本： 7周6根阴线，玩够了吧
情感分值： 0
情感倾向：中性
情感分析文本： 要变盘了
情感分值： 1
情感倾向：积极
情感分析文本： 而且按目前的走势，散户先躲一躲总是没错的，不要老想
情感分值： -2
情感倾向：消极
情感分析文本： 无裤叉穿了[怒][哭]
情感分值： 0
情感倾向：中性
情感分析文本： em应该控制一下评论里做广告骗人的……要是真能挣钱，
情感分值： -1
情感倾向：消极
情感分析文本： 今年元旦了，没希望了，跑得越早越好
情感分值： 2
情感倾向：积极
情感分析文本： 昨天一则消息比较受关注，就是著名的多头“李大霄”开
情感分值： 1
情感倾向：积极
情感分析文本： 机会
情感分值： 4
情感倾向：积极
情感分析文本： 小猪刚从图书馆回来，今天发的有点晚了，沪深300指数
情感分值： 2
情感倾向：积极
情感分析文本： 日线DIFF继续向上！明天开始关注日线0轴是否突破，突
情感分值： -1
情感倾向：消极
情感分析文本： 主板混的不如创业板，你连主板都不如
情感分值： 29
情感倾向：积极
情感分析文本： 芯片大跌，据说是国家大基金减持，其实就是涨的太高了
情感分值： 0
情感倾向：中性
情感分析文本： 无非一个信念始终一定要绿
情感分值： 2
情感倾向：积

情感分值： 0
情感倾向：中性
情感分析文本： 又看了一遍《不能说的秘密》，哎呀，我要是能去20年后
情感分值： 0
情感倾向：中性
情感分析文本： [图片]
情感分值： 25
情感倾向：积极
情感分析文本： 本周，各大指数全线上涨，创业板指数表现最为出色，涨
情感分值： 0
情感倾向：中性
情感分析文本： 白酒行业出了新瓜......
情感分值： 0
情感倾向：中性
情感分析文本： 我有预感6月30日这一天收盘所有股票是红盘报收，虽然
情感分值： 0
情感倾向：中性
情感分析文本： 我有预感4月30日这一天所有股票是红盘报收，虽然涨幅
情感分值： 4
情感倾向：积极
情感分析文本： 今天沪深300指数又涨了，还是大涨呢，估计凡事有点仓
情感分值： 1
情感倾向：积极
情感分析文本： 3月25日预测：3月底或者4月初有反弹。4月2日预测：4月
情感分值： 2
情感倾向：积极
情感分析文本： 今天出了上涨波段中的大阳线！大阳线是加速期，耐心等
情感分值： 4
情感倾向：积极
情感分析文本： 白酒股迎来强势反弹主力大手笔加仓五粮液：今日白酒
情感分值： 0
情感倾向：中性
情感分析文本： 现在出手，周一转
情感分值： -1
情感倾向：消极
情感分析文本： 我说做空沪深300就是做空，我就做保守的义务方，我赚
情感分值： 0
情感倾向：中性
情感分析文本： 哎，比不过喝酒的！
情感分值： 24
情感倾向：积极
情感分析文本： 沪深300在从现在算到七八月份会大幅拉升，高点在6000
情感分值： 0
情感倾向：中性
情感分析文本： 冲冲冲
情感分值： 0
情感倾向：中性
情感分析文本： 极品极品
情感分值： 1
情感倾向：积极
情感分析文本： 成绩不错哦
情感分值： 1
情感倾向：积极
情感分析文本： 快下去吧，别装了
情感分值： 3
情感倾向：积极
情感分析文本： A股机会早知道：氨纶价格再创新高相关公司业绩可期
情感分值： 0
情感倾向：中性
情感分析文本： 为什么不能发行QF指数ETF和社保指数E丅F？？？是不能
情感分值： 2
情感倾向：积极
情感分析文本： 今天三大指数高开冲高回落，上证最高上冲到3570，最低
情感分值： 0
情感倾向：中性
情感分析文本： 好久没来银华日利发帖啦，今天沪深300涨了，所以明天
情感分值： 0
情感倾向：中性
情感分析文本： 继续

情感分值： 0
情感倾向：中性
情感分析文本： 明天再来一中阴就到位了
情感分值： -14
情感倾向：消极
情感分析文本： 涨幅实在太小了，躺平不动，佛系小猪，哈哈哈$沪深300
情感分值： 2
情感倾向：积极
情感分析文本： 300指数今天缩量，小幅波动，贴线收小阳，站在20均线
情感分值： 1
情感倾向：积极
情感分析文本： 行情没有其他选择了，要么下来形成15分钟底部背离之后
情感分值： 6
情感倾向：积极
情感分析文本： 低开上攻，力道略有不足，没啥说的，多看少动。新版基
情感分值： 1
情感倾向：积极
情感分析文本： 甘化可能
情感分值： -2
情感倾向：消极
情感分析文本： 这整天跌跌不休，没完没了[亏大了]
情感分值： 0
情感倾向：中性
情感分析文本： 现在的指数就是今天的底，明天怎么走不知道。
情感分值： 0
情感倾向：中性
情感分析文本： 一个奇葩的市场
情感分值： 2
情感倾向：积极
情感分析文本： boll线喇叭口收紧。暴风雨前的平静。准备突破了
情感分值： -1
情感倾向：消极
情感分析文本： 上不上下不下的，我就横着，哎，就是玩儿~
情感分值： -9
情感倾向：消极
情感分析文本： 指数在6月19日之前是绝对跌不动的，肯定有1根大阳线。
情感分值： -1
情感倾向：消极
情感分析文本： 这货怎么天天跌
情感分值： 2
情感倾向：积极
情感分析文本： 股指真的是BT中的航母！
情感分值： -2
情感倾向：消极
情感分析文本： 新一轮跌跌跌模式开启
情感分值： 0
情感倾向：中性
情感分析文本： 今天三大指数冲高回落，全线收绿，上证早盘一度被券商
情感分值： 2
情感倾向：积极
情感分析文本： 涨价近十倍，一箱仍难求！低估值高增长航运港口概念
情感分值： 1
情感倾向：积极
情感分析文本： A股机会早知道：龙蟒佰利上调各型号钛白粉500元/吨
情感分值： 1
情感倾向：积极
情感分析文本： 美股将在光明顶上完成多空易位
情感分值： 2
情感倾向：积极
情感分析文本： 集体股评都说最后调整，要大涨了，主力会配合吗
情感分值： -2
情感倾向：消极
情感分析文本： 这是要阴跌吖
情感分值： -1
情感倾向：消极
情感分析文本： 15分钟钝化消失了。明天，继续关注15分钟底背离，市场
情感分值： 2
情感倾向：积极
情感分析文本： 白酒集体扑街，上

...

情感倾向：消极
情感分析文本： 熊市来了
情感分值： -4
情感倾向：消极
情感分析文本： 我就想知道，为何300跌的比上证还多？
情感分值： 1
情感倾向：积极
情感分析文本： 不要跑
情感分值： 1
情感倾向：积极
情感分析文本： 不要跑
情感分值： -1
情感倾向：消极
情感分析文本： 终于还是没能守住5390，只能寄希望于下周之内不要再出
情感分值： -2
情感倾向：消极
情感分析文本： 又不行了，没有上涨动力
情感分值： -6
情感倾向：消极
情感分析文本： 洗盘意思意思就可以了，别太过分哈！！
情感分值： 0
情感倾向：中性
情感分析文本： 最难熬的日子来了
情感分值： 1
情感倾向：积极
情感分析文本： 用周K线的角度看大盘，各中长期均线已依序排列，发散
情感分值： 1
情感倾向：积极
情感分析文本： 空头释放完能量了，又是多头恢复信心的时候了。
情感分值： 0
情感倾向：中性
情感分析文本： 如果春节前能跌到位，也许可以期待一下春节红包。
情感分值： -1
情感倾向：消极
情感分析文本： 最近老超顶
情感分值： 0
情感倾向：中性
情感分析文本： 402股预计全年净利润增幅翻倍：统计显示，截至1月28
情感分值： 10
情感倾向：积极
情感分析文本： 一年定投沪深300绝对不亏
情感分值： 0
情感倾向：中性
情感分析文本： 不要慌，洗盘而已，国家队闪电战洗盘罢了。
情感分值： 0
情感倾向：中性
情感分析文本： 买，买，买
情感分值： 0
情感倾向：中性
情感分析文本： 涨少跌多
情感分值： 7
情感倾向：积极
情感分析文本： 5390很关键，是双头颈线位，如果破位，乐观支撑在5360
情感分值： 1
情感倾向：积极
情感分析文本： 机构调研丨中文在线获49家机构集体调研，关注科技巨
情感分值： 1
情感倾向：积极
情感分析文本： 不是你的错
情感分值： -8
情感倾向：消极
情感分析文本： 年报预警汇总丨这些公司2020年业绩大幅亏损，大唐电
情感分值： -7
情感倾向：消极
情感分析文本： t+0对庄家更有利，不出三个月就能把散户全玩死。
情感分值： 2
情感倾向：积极
情感分析文本： 早日开通T+0，当天的涨跌当天清，机构抱不了团，散户
情感分值： 2
情感倾向：积极
情感分析文本： 北向资金大举增持的业绩预增股名单来了：业绩向好叠
情感分值

情感分值： 1
情感倾向：积极
情感分析文本： 茅台新高在即
情感分值： -2
情感倾向：消极
情感分析文本： 害怕呀，看长的害怕
情感分值： 0
情感倾向：中性
情感分析文本： 此刻户型300成交额超过上证成交额！！！(2021年1月12
情感分值： 1
情感倾向：积极
情感分析文本： 真牛啊，沪深300要上天
情感分值： 3
情感倾向：积极
情感分析文本： 三大股指悉数翻红，汽车领跌、军工股上涨：A股三大
情感分值： 0
情感倾向：中性
情感分析文本： 注册制就是杀小盘股？
情感分值： 1
情感倾向：积极
情感分析文本： 年报预告频出，首份高送转来袭！
情感分值： 2
情感倾向：积极
情感分析文本： 沪深股通现身16股龙虎榜净买入英科医疗超2亿元：win
情感分值： 1
情感倾向：积极
情感分析文本： 172家公司业绩将翻番，这些公司业绩增长上限超过100
情感分值： 0
情感倾向：中性
情感分析文本： 这个指数怎么买，哪里可以买
情感分值： 1
情感倾向：积极
情感分析文本： 今天居然红了
情感分值： 0
情感倾向：中性
情感分析文本： 明天大盘调到位，上证3500点就是买点。
情感分值： -8
情感倾向：消极
情感分析文本： 白酒板块大幅下挫10股遭主力抛售超亿元：今日白酒板
情感分值： 0
情感倾向：中性
情感分析文本： 机会OR风险
情感分值： 2
情感倾向：积极
情感分析文本： 10个点以内回调都正常，这是机会
情感分值： -1
情感倾向：消极
情感分析文本： 咋回事啊，除了茅台都不带劲
情感分值： 1
情感倾向：积极
情感分析文本： 周五被吓出去了，现在上车鹏华酒160632还来的及吗
情感分值： -5
情感倾向：消极
情感分析文本： 市场回调也不慌，咱持仓很稳[微笑]~公布下最新持仓：
情感分值： -1
情感倾向：消极
情感分析文本： 没到最后别这么急着下定论
情感分值： 2
情感倾向：积极
情感分析文本： 车辆起步，请坐稳扶好
情感分值： 0
情感倾向：中性
情感分析文本： 兄弟们，鹏华酒160632懂得都懂哦
情感分值： 4
情感倾向：积极
情感分析文本： 过几天买价钱更好
情感分值： 1
情感倾向：积极
情感分析文本： 周五追在最高点，我就纳闷了，主力看上我手上我2万块
情感分值： 1
情感倾向：积极
情感分析文本： 这是近两年各大指数的表现

情感分值： 1
情感倾向：积极
情感分析文本： 这样做，你才会赢
情感分值： 0
情感倾向：中性
情感分析文本： 立着牌坊耍流氓
情感分值： -1
情感倾向：消极
情感分析文本： 弱。。。。
情感分值： 0
情感倾向：中性
情感分析文本： 大家赶紧卖啊，吧北上资金闷杀了
情感分值： 2
情感倾向：积极
情感分析文本： 重大改革来了
情感分值： 5
情感倾向：积极
情感分析文本： 现在指数是神马回事？看指数还以为是大牛市，手里股票
情感分值： 0
情感倾向：中性
情感分析文本： 别急，市场还没走出来
情感分值： -1
情感倾向：消极
情感分析文本： 10月跑路的我拍断大腿
情感分值： 4
情感倾向：积极
情感分析文本： 还好我是定投
情感分值： 2
情感倾向：积极
情感分析文本： 新能源车涨，利好茅台
情感分值： -8
情感倾向：消极
情感分析文本： 管他后期怎么走，不操作就是最好的操作
情感分值： 0
情感倾向：中性
情感分析文本： 搞什么战略科技？我们有酒！
情感分值： -5
情感倾向：消极
情感分析文本： 煤炭钢铁被打的很惨
情感分值： 1
情感倾向：积极
情感分析文本： 酒科技牛
情感分值： 0
情感倾向：中性
情感分析文本： 三家巨头计划在印尼该国投资建厂
情感分值： 2
情感倾向：积极
情感分析文本： 中证食品、中证酒等盘中又创历史新高了
情感分值： 0
情感倾向：中性
情感分析文本： 最近趋势应该是向上的
情感分值： 0
情感倾向：中性
情感分析文本： 下跌总是如此流畅
情感分值： 0
情感倾向：中性
情感分析文本： 赌十块钱，尾盘绝对飘绿
情感分值： 1
情感倾向：积极
情感分析文本： 雄起到收盘[拜神]
情感分值： 0
情感倾向：中性
情感分析文本： 成交量不足，全天预计就1亿左右，低于昨天的1.2亿，过
情感分值： -4
情感倾向：消极
情感分析文本： 继续爬坡行情，时间会比较长，不会一蹴而就。每天盘中
情感分值： 5
情感倾向：积极
情感分析文本： 挂了4.9买个2成仓，今天跌2个点有没有戏？
情感分值： 2
情感倾向：积极
情感分析文本： 对酿酒的技术浅见，简单有效，不久的将来应该能看到，
情感分值： 1
情感倾向：积极
情感分析文本： 本周最新行业估值
情感分值： 2
情感倾向：积极
情感分析文本： 晚间继续分享：筛选主动基金，另一个主要因子

情感分值： 1
情感倾向：积极
情感分析文本： 不跌还让你亏才是高
情感分值： 0
情感倾向：中性
情感分析文本： 恒久科技又跌停什么情况
情感分值： -4
情感倾向：消极
情感分析文本： 中国牛跑了吗？
情感分值： 1
情感倾向：积极
情感分析文本： 现在沪深300的平均市盈率是15倍。2015年顶部的市盈率1
情感分值： 0
情感倾向：中性
情感分析文本： 明年大牛
情感分值： 0
情感倾向：中性
情感分析文本： 牛市的味道消灭低估值
情感分值： 1
情感倾向：积极
情感分析文本： 一会拉钢铁，一会拉猪，拉银行，拉地产，拉券商，还拉
情感分值： 0
情感倾向：中性
情感分析文本： 这是加速赶顶
情感分值： 2
情感倾向：积极
情感分析文本： [cp]谨防冲高后的大跌：1127盘前：今天盘后对沪深
情感分值： -1
情感倾向：消极
情感分析文本： 与2017年底的行情惊人的类似
情感分值： 4
情感倾向：积极
情感分析文本： 生拉硬拽，有意思吗？
情感分值： 2
情感倾向：积极
情感分析文本： 神奇的指数又红了
情感分值： 0
情感倾向：中性
情感分析文本： 中矿资源：英雄本铯
情感分值： -2
情感倾向：消极
情感分析文本： 三只黑乌鸦看空
情感分值： 1
情感倾向：积极
情感分析文本： 千万别红啊，我要赌输了
情感分值： 0
情感倾向：中性
情感分析文本： 什么时候见底？
情感分值： 0
情感倾向：中性
情感分析文本： 又被机构玩弄
情感分值： -1
情感倾向：消极
情感分析文本： 拉稀吧，
情感分值： -1
情感倾向：消极
情感分析文本： 回踩4925做低
情感分值： 0
情感倾向：中性
情感分析文本： 逢高做低
情感分值： 5
情感倾向：积极
情感分析文本： 抛吧，庄控带来的股灾，你还真以为是牛市
情感分值： 5
情感倾向：积极
情感分析文本： 驴深300指数又要调整了吗？记得上次调整不到五天啊！[
情感分值： -1
情感倾向：消极
情感分析文本： 炒股穷一生，销户富三代！
情感分值： 0
情感倾向：中性
情感分析文本： 昨天收盘卖空仓，今天没有波动，最近公开的点位已经完
情感分值： 0
情感倾向：中性
情感分析文本： 明天还得降！
情感分值： 1
情感倾向：积极
情感分析文本： [cp]上涨暂告一段落：1125收盘：一根大阴线，行情
情感分值： 0
情感

情感分析文本： 9.10：本人9月以来确实跑赢了大盘，但也不是那种很
情感分值： 2
情感倾向：积极
情感分析文本： 净小单一上来，就开始跌
情感分值： 1
情感倾向：积极
情感分析文本： 盘面解析：早盘高开低走，是收到外部因素影响的高开，
情感分值： 1
情感倾向：积极
情感分析文本： 今天大跌呀$沪深300(SH000300)$
情感分值： 0
情感倾向：中性
情感分析文本： pym
情感分值： -1
情感倾向：消极
情感分析文本： 现在是两点半加速时间，如果多头再组织不起有效反击，
情感分值： 1
情感倾向：积极
情感分析文本： 又到了每天两点的加速期。话说纳指已经企稳，其实今天
情感分值： 1
情感倾向：积极
情感分析文本： 恭喜恭喜恭喜恭喜恭喜
情感分值： 1
情感倾向：积极
情感分析文本： 沪深300都超过当年5000点了
情感分值： 0
情感倾向：中性
情感分析文本： 收工了，心脏受不了，操的卖白粉的心，赚着卖白菜的钱
情感分值： 1
情感倾向：积极
情感分析文本： 没出低点
情感分值： 0
情感倾向：中性
情感分析文本： 骗我接盘，门都没有
情感分值： 2
情感倾向：积极
情感分析文本： 今天逢高干空，基本面支持[大笑]
情感分值： 3
情感倾向：积极
情感分析文本： 主力大资金大逃亡，小心
情感分值： 0
情感倾向：中性
情感分析文本： 海通证券的蟳狱根，你说说看，这是牛市第几浪
情感分值： -11
情感倾向：消极
情感分析文本： 跌太少了今天起码跌1万点才合理
情感分值： 0
情感倾向：中性
情感分析文本： 好不了
情感分值： 1
情感倾向：积极
情感分析文本： 到位啦，该涨了
情感分值： 2
情感倾向：积极
情感分析文本： 老乡别走，有利好在路上。
情感分值： -2
情感倾向：消极
情感分析文本： 低风险
情感分值： 0
情感倾向：中性
情感分析文本： 受美股影响明天闷杀今天买进去的
情感分值： 0
情感倾向：中性
情感分析文本： 一个猜想，明天是放量盘整或者缩量大幅拉涨，如果明天
情感分值： 1
情感倾向：积极
情感分析文本： 全靠拉金融三傻，无量拉升骗炮，逃命吧大家
情感分值： -1
情感倾向：消极
情感分析文本： 下跌中继而已
情感分值： 1
情感倾向：积极
情感分析文本： 硬拉三傻骗炮，逃命机会
情感分值： 0
情感倾向：中性
情感分析